# Higher orders of approximation


In this notebook, we see how the use of higher degree polynomials within mesh elements results in higher orders of accuracy when approximating smooth functions. This is the primary reason for going beyond the lowest order Lagrange finite element space.

Consider the **degree $p$ Lagrange finite element space**
$
V_h^p = \{ v: v $ is continuous and $
v|_K $ is a polynomial of degree $\le p$
on each mesh element $K\}.$ In later lectures, we will prove the Bramble-Hilbert Lemma and its  corollary 
$$
\inf_{v_h \in V_h^p} \| u - v_h\|_{H^1(\Omega)}
\le C h^\ell \| u\|_{H^{\ell+1}(\Omega)}
$$
for $0\le \ell \le p$ where $h$ is the maximal element diameter and $C$ is a constant independent of $h$ (element sizes) but  dependent on the *shapes of the elements* (or angles) in the mesh. Note that the $u$ must be regular enough for the above estimate to hold (namely, the derivatives of $u$ of order upto $\ell+1$ must be square integrable). Thus when $u$ is smooth, we should be able to see up to $O(h^p)$-rate of approximation in the $H^1(\Omega)$ norm for the best approximation error. 

## Elliptic projection 

Let $P_h : H^1(\Omega) \to V_h^p$ denote the orthogonal projector into $V_h^p$ in the $(\cdot, \cdot)_{H^1(\Omega)}$ innerproduct.
Since 
$$
\inf_{v_h \in V_h^p} \| u - v_h\|_{H^1(\Omega)}
 =  \| u - P_h u \|_{H^1(\Omega)},
$$
we can study the above infimum, or  the $H^1(\Omega)$ **best approximation error**  (the $H^1(\Omega)$ **BAE**), 
by studying the numerical convergence of $u - P_h u$  in the $H^1(\Omega)$-norm.


From the properties of orthogonal projections, we immediately see that $P_h u$ is the unique function in 
$V_h^p$ satisfying 
$$
\langle{P_h u, v_h\rangle}_{H^1(\Omega)} = 
\langle{u, v_h\rangle}_{H^1(\Omega)} 
$$
for all $v_h \in V_h^p$, where $\langle{\cdot, \cdot\rangle}_{H^1(\Omega)} $ denotes the inner product of $H^1(\Omega)$, 
 or equivalently, 
 
$$
\int_\Omega \nabla(P_h u) \cdot \nabla v_h + P_h u \; v_h
=
\int_\Omega \nabla u \cdot \nabla v_h + u \, v_h.
$$

This equation for $P_h u$ is solved in the next function, designed using the same type of code you have seen in the previous tutorials. (Projections like $P_h u$ are sometimes called elliptic projections as they are solutions of elliptic PDEs.)

In [ ]:
from ngsolve import grad, dx, CF

def Project(u, V):
    """
    Input: u as a CF.
    Output: H^1-projection P_h u as a GF.
    """    
    gradu = CF((u.Diff(x), u.Diff(y)))
    Pu = ng.GridFunction(V)
    w, v = V.TnT()
    a = ng.BilinearForm(grad(w)*grad(v)*dx + w*v*dx, symmetric=True)
    b = ng.LinearForm(gradu*grad(v)*dx + u*v*dx)
    
    with ng.TaskManager():  # Exploit task parallelism if possible
        a.Assemble()
        b.Assemble()
        Pu.vec.data = a.mat.Inverse(inverse='sparsecholesky') * b.vec
    return Pu

## Successive mesh refinements

Studying the BAE over a collection of meshes is simplified if element *shapes* (not sizes) do not vary too much over the collection. A sequence of meshes where the element angles never change from  those of the initial mesh is obtained by a simple *uniform refinement*. This is illustrated in the sequence of mesh images below.

In [ ]:
import ngsolve as ng
ng.ngsglobals.msg_level=2
from netgen.geom2d import unit_square
from ngsolve.webgui import Draw
mesh = ng.Mesh(unit_square.GenerateMesh(maxh=0.3))
Draw(mesh)

In [ ]:
mesh.ngmesh.Refine(); Draw(mesh);

In [ ]:
mesh.ngmesh.Refine(); Draw(mesh);

Note that this uniform refinement process, when started on any coarse mesh, does not generate any new angles within the new elements, i.e., each triangular element of this infinite sequence  of meshes is similar (in the geometric sense) to one of the triangles in the initial coarse mesh. 

## BAE on uniform refinements

We compute the elliptic projection on a sequence of uniform refinements, starting from a very corase mesh,  using the function below.

In [ ]:
from ngsolve import Mesh, InnerProduct, Integrate, sqrt
import numpy as np


def ProjectOnSuccessiveRefinements(u, p=1, hcoarse=1, nrefinements=6):
    """ Solve the projection problem on a sequence of succesively
    refined meshes. """
    
    Pus = []; errors = []; meshes = []; ndofs = []
    mesh = ng.Mesh(unit_square.GenerateMesh(maxh=hcoarse))
    gradu = CF((u.Diff(x), u.Diff(y)))
    
    for ref in range(nrefinements): 
        V = ng.H1(mesh, order=p)

        Pu = Project(u, V) 
        
        sqrerr = (Pu - u)**2 + (grad(Pu) - gradu)**2

        Pus.append(Pu) 
        errors.append(sqrt(Integrate(sqrerr, mesh)))
        meshes.append(ng.Mesh(mesh.ngmesh.Copy()))
        ndofs.append(V.ndof)
        
        mesh.ngmesh.Refine()

    return Pus, np.array(errors), ndofs, meshes

We will select an infinitely smooth function $u$ for the initial experiments (and you will experiment with less regular functions in exercises). We select a smooth $u$ with some oscillations  (as otherwise the approximation errors rapidly become too close to machine precision).

In [ ]:
from ngsolve import cos, x, y

u = cos(10*x*y)

We compute its BAE in the lowest order Lagrange finite element space for uniform refinements by invoking the above function. 

In [ ]:
Pus, es, ns, _ = ProjectOnSuccessiveRefinements(u, p=1, nrefinements=8)
errors = {1: es}; ndofs = {1: ns}
es

Clearly this sequence of errors are decreasing, approximately halving at each step.

## Estimate rate of convergence 

We want to estimate at what rate the errors on successive refinements, stored in the list `es`, go to zero.  Let $e_i$ denote the $i$th element of the list.


Specifically, what we would like to determine is the number $r$, the **rate of covergence**, such that the errors are bounded by $O(h^r).$
If the sequence $e_i$ goes to zero like $O(h_i^r)$, then  since 
$$
h_i = \frac{h_0}{2^i}
$$ 
we should see $e_{i+1} / e_i  \sim O(2^{-r})$. Hence to estimate $r$, we compute 
$$
\log_2 \frac{e_{i+1}}{ e_i}.
$$
These logarithms are computed and tabulated together with the error numbers using the formatting function below.

In [ ]:
from prettytable import PrettyTable

def tabrate(name, dat, h0=1):
    """Inputs: 
        name = Name for second (error norm) column, 
        dat = list of error data on successive refinements
        log2h0inv = log2(1/h0), where h0 is coarsest meshsize.
    """
    col = ['h', name, 'rate']
    t = PrettyTable()
    h0col = ['%g'%h0]
    t.add_column(col[0], h0col + [h0col[0] + '/' + str(2**i) 
                                  for i in range(1, len(dat))])
    t.add_column(col[1], ['%.8f'%e for e in dat])
    t.add_column(col[2], ['*'] + \
                 ['%1.2f' % r 
                  for r in np.log(dat[:-1]/dat[1:])/np.log(2)])
    print(t)

#### $p=1$ case

In [ ]:
tabrate('H1norm(Pu-u)', es)

#### $p=2$ case

In [ ]:
Pus, es, ns, _ = ProjectOnSuccessiveRefinements(u, p=2, nrefinements=7)
errors[2] = es; ndofs[2] = ns
tabrate('H1norm(Pu-u)', es)

#### $p=3$ case

In [ ]:
Pus, es, ns, _ = ProjectOnSuccessiveRefinements(u, p=3)
errors[3] = es; ndofs[3] = ns
tabrate('H1norm(Pu-u)', es)

#### $p=4$ case 

In [ ]:
Pus, es, ns, _ = ProjectOnSuccessiveRefinements(u, p=4)
errors[4] = es; ndofs[4] = ns
tabrate('H1norm(Pu-u)', es)

#### $p=5$ case

In [ ]:
Pus, es, ns, _ = ProjectOnSuccessiveRefinements(u, p=5)
errors[5] = es; ndofs[5] = ns
tabrate('H1norm(Pu-u)', es)

## Accuracy vs. degrees of freedom

During the above computations, we have also stored $\dim(V_h^p)$, or the **number of degrees of freedom**. This is useful when trying to gauge how much bang for the buck we obtain when using various meshes and various orders $p$.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

for p in range(1, 6):
    plt.loglog(ndofs[p], errors[p], '.-', label='p=%d'%p)

plt.xlabel('Degrees of Freedom')
plt.ylabel('$H^1$ best approximation error'); plt.legend();

We see from these results that for the smooth `u` under consideration,  the accuracy of its best approximation, measured *per* degree of freedom increases (quite dramatically) when using higher $p$.

<hr>

### Exercise

Modify the above computations to determine the rate of convergence of the $L^2(\Omega)$ BAE 
$$
\inf_{v_h \in V_h^p} \| u - v_h\|_{L^2(\Omega)}
$$
(i.e., keeping the same spaces, meshes and degrees, but changing the norm in which the error is measured).

### Exercise 

Prove that 
$$
\text{dim}(V_h^p) = n_V + (p-1) n_E + \frac 1 2 (p-1) (p-2) n_K
$$
where $n_V, n_E,$ and $n_K$ denote the number of vertices, edges, and triangles of the mesh. 

This dimension for the meshes and the $p$-values we used in the above computations are in  `ndofs`, if you think it is illustrative to compare it with the above formula:

In [ ]:
ndofs

### Exercise

Let $\Omega$ be the L-shaped domain shown:

In [ ]:
#  (-1,1)                       (1,1)
#       +----------------------+  
#       |                      |
#       |                      | 
#       |        (0,0)         |
#       |           +----------+ (1,0)
#       |           |     
#       |           | 
#       |           |
#       +-----------+
#  (-1,-1)       (0,-1)

Compute the rates of convergence of $H^1(\Omega)$ BAE and $L^2(\Omega)$ BAE, for various degrees $p$, for the following two functions expressed in polar coordinates $r = \sqrt{x^2 +y^2}$, $\varphi=\tan^{-1}(y/x)$ on the above domain:
$$
u = r^2 \sin(2\varphi/3)
$$
and 
$$
u = r^{2/3} \sin(2\varphi/3).
$$
(Be careful to not get into trouble with the branch cut of arctan while setting $\varphi$ in your computations.)


<hr>

## Postscript

Did you get the message from the exercises on how regularity influences convergence rates? 
Here is a poem from the 1978 Finite Element Circus that will reinforce the message.

> There once was a fellow named Dare. <br>
> Who approximated PDEs with great care. <br>
> But the solutions were rough <br>
> And the problems were tough, <br>
> So he only got $O(h^2).$